In [10]:
import langgraph
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import AnyMessage,HumanMessage,AIMessage,SystemMessage,ToolMessage
from typing import TypedDict,List,Annotated
import operator
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os
from langgraph.graph import StateGraph,END

In [7]:
load_dotenv()
os.environ["OPENAI_API_KEY"]=os.getenv("OPEN_API_KEY2")
os.environ["TAVILY_API_KEY"]=os.getenv("TAVILY_API_KEY")

In [64]:
tool = TavilySearchResults(max_results=2)

In [65]:
class AgentState(TypedDict):
    """ AgentState` is a TypedDict that represents the state of an agent.
    # The `messages` key holds a list of `AnyMessage` objects.
    # The `Annotated` type is used here to associate the `messages` list with
    # an additional metadata operator (`operator.add`), which suggests that new 
    # messages might be appended using this operator """
    messages: Annotated[list[AnyMessage], operator.add]

In [69]:
memory = MemorySaver()

In [70]:
class Agent:
    def __init__(self, model, tools, checkpointer, system=""):
        """
        Initializes an Agent with a model, tools, a checkpointer, and an optional system message.

        Parameters:
        - model: The language model that interacts with the agent (e.g., OpenAI's GPT model).
        - tools: A list of tools the agent can use, which will be stored in a dictionary by their name.
        - checkpointer: Used to save and restore the state of the graph, ensuring continuity.
        - system: An optional system message that can be prepended to the conversation context.

        The agent's behavior is modeled as a graph (`StateGraph`), where nodes represent
        different functions (`call_openai`, `take_action`), and edges define the flow between them.
        The graph starts at the "llm" node and moves conditionally based on whether the agent
        needs to take an action (e.g., invoke a tool).
        """
        self.system = system
        
        # Initialize the state graph with the structure of the agent's state (AgentState)
        graph = StateGraph(AgentState)
        
        # Add nodes (functions) to the graph:
        # "llm" node invokes the OpenAI model and updates the state with a new message.
        # "action" node handles tool invocation based on the latest message.
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        
        # Conditionally add edges from "llm" to "action" based on whether an action is required.
        # If the latest message includes tool calls, go to "action"; otherwise, end.
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        
        # Add an edge from "action" back to "llm", enabling continuous conversation or actions.
        graph.add_edge("action", "llm")
        
        # Set the starting point of the graph as "llm".
        graph.set_entry_point("llm")
        
        # Compile the graph with a checkpointer for saving and restoring agent state.
        self.graph = graph.compile(checkpointer=checkpointer)
        
        # Convert tools into a dictionary for easy access during execution (by tool name).
        self.tools = {t.name: t for t in tools}
        
        # Bind the model to the tools so it can call them when necessary.
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        """
        Calls the OpenAI model with the current message state.

        Parameters:
        - state: The current state of the agent, containing messages.

        If a system message is defined, it will be prepended to the list of messages.
        The model is then invoked with the messages, and a new message is returned,
        which gets added back to the state.
        """
        # Retrieve messages from the current state.
        messages = state['messages']
        
        # If a system message is provided, prepend it to the messages.
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        
        # Call the model with the messages and receive a new message.
        message = self.model.invoke(messages)
        
        # Return the updated state with the new message added.
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        """
        Checks whether the last message in the state requires an action (tool invocation).

        Parameters:
        - state: The current state of the agent, containing messages.

        Returns:
        - True if the last message includes tool calls, False otherwise.
        """
        # Get the latest message in the state.
        result = state['messages'][-1]
        
        # Check if the message contains any tool calls.
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        """
        Invokes the necessary tools based on the last message's tool calls.

        Parameters:
        - state: The current state of the agent, containing messages.

        The method loops through the list of tool calls, invokes each tool with its
        respective arguments, and appends the result back to the messages in the form
        of a ToolMessage.
        """
        # Retrieve tool calls from the latest message in the state.
        tool_calls = state['messages'][-1].tool_calls
        
        # Initialize a list to hold the results of the tool invocations.
        results = []
        
        # Loop through each tool call, invoke the respective tool, and store the result.
        for t in tool_calls:
            print(f"Calling: {t}")
            
            # Invoke the tool by its name, passing the required arguments.
            result = self.tools[t['name']].invoke(t['args'])
            
            # Create a ToolMessage with the result and add it to the results list.
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        
        # Log that the tool action is complete and the agent is returning to the model.
        print("Back to the model!")
        
        # Return the updated state with the new tool result messages.
        return {'messages': results}


In [71]:
model = ChatOpenAI(model="gpt-3.5-turbo")
prompt="""You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
abot = Agent(model=model,tools=[tool],checkpointer=memory,system=prompt)

In [96]:
messages = [HumanMessage(content="What is the weather in sf?")]

In [97]:
thread = {"configurable": {"thread_id": "1"}}

In [105]:
result=abot.graph.invoke(
    {"messages": [HumanMessage(content="i also mention sf but why you guess karachi")]},
    config={"configurable": {"thread_id": 42}}
)

In [106]:
result["messages"][-1].content

"I apologize for the confusion. You mentioned both San Francisco (sf) and Karachi in your queries. I associated Karachi with your location based on the context of your questions. If you have any specific questions or topics you'd like to discuss, feel free to let me know!"